In [2]:
import csv, string, re, time, nltk
import numpy as np
import pandas as pd
from collections import defaultdict

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from nltk.corpus import stopwords

#from nltk.tokenize import TweetTokenizer
#from sklearn.model_selection import KFold, cross_val_score
#from sklearn.ensemble import RandomForestClassifier

nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrb88\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
def tsv_parser(filename, delimiter='\t', encoding='utf-8'):
    with open(filename, 'r', encoding=encoding) as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter=delimiter)
        df = pd.DataFrame(data=list(tsv_reader))
    return df

# Using function to extract the data from the files
print('Parsing data from TSV file...')
start_time = time.time()
df = tsv_parser('train.tsv')
print(f'Done. Took {time.time()-start_time:.2f}s')

Parsing data from TSV file...
Done. Took 11.56s


In [4]:
# Use first row for column names, then drop it from the data
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))

df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
1,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,,10.0,1,No description yet
2,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
3,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
4,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,,35.0,1,New with tags. Leather horses. Retail for [rm]...
5,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,,44.0,0,Complete with certificate of authenticity


In [5]:
y = df.loc[:,'price']
y.head()

1    10.0
2    52.0
3    10.0
4    35.0
5    44.0
Name: price, dtype: object

In [7]:
X = df
del X['price']
X.head()

,train_id,name,item_condition_id,category_name,brand_name,shipping,item_description
1,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,,1,No description yet
2,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,0,This keyboard is in great condition and works ...
3,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,1,Adorable top with a hint of lace and a key hol...
4,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,,1,New with tags. Leather horses. Retail for [rm]...
5,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,,0,Complete with certificate of authenticity


In [6]:
regex = re.compile('[%s]' % re.escape(string.punctuation))

# def format_string(s):
#     """
#     Convert string to lowercase , replace hyphens with a space, and remove punctuation
#     """
#     s = s.lower()
#     s = s.replace('-',' ')
#     return regex.sub('', s)

def format_tweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
        #     # Remove Punctuation and split 's, 't, 've with a space for filter
        #     tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove punctuation
    tweet = regex.sub('', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet